This version of the code focuses on trying to determine how the time taken for an insert, select, update or delete operation varies regarding the size of the query on an indexed database previously filled up with 1000 data.
Authors: MAIRE Maxime & VLAYKOV Mathéo

In [1]:
import redis
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from os.path import exists

In [2]:
def create_sqlite_database(con_init, cur_init):
    """As the sqlite database will not be modified in this code, we create it only once and in this function.
        This sqlite database is an extract of the original mental_health.sqlite.

    Args:
        con_init (Connection): The connection to the original database
        cur_init (Cursor): The cursor to the original database
    """
    if not exists("mental_health_v2.sqlite"):
        con = sqlite3.connect("mental_health_v2.sqlite")
        cur = con.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS survey (SurveyID INTEGER PRIMARY KEY, description TEXT);")
        cur.execute("CREATE TABLE IF NOT EXISTS question (questiontext TEXT, QuestionID INTEGER PRIMARY KEY);")
        cur.execute("CREATE TABLE IF NOT EXISTS answer (AnswerText TEXT, SurveyID INTEGER, UserID INTEGER, QuestionID INTEGER, PRIMARY KEY(SurveyID, UserID, QuestionID));")
        con.commit()

        res = cur_init.execute("SELECT * from survey ;")
        result = res.fetchall()
        res2 = cur_init.execute("SELECT * from question ;")
        result2 = res2.fetchall()
        res3 = cur_init.execute("SELECT * from answer LIMIT 1000;")
        result3 = res3.fetchall()

        for res in result:
            cur.execute("INSERT INTO survey (SurveyID, description) VALUES (?, ?)", res)

        for res in result2:
            cur.execute("INSERT INTO question (questiontext, QuestionID) VALUES (?, ?)", res)

        for res in result3:
            cur.execute("INSERT INTO answer (AnswerText, SurveyID, UserID, QuestionID) VALUES (?, ?, ?, ?)", res)

        con.commit()

In [3]:
def launch_all(nb_data):
    print("##################################################")
    print("OPERATIONS SUR",nb_data,"DONNEES ")
    print("##################################################\n\n")
    start_time = datetime.now().timestamp()
    r = redis.StrictRedis(host='localhost', port=6379, db=0)
    end_time = datetime.now().timestamp()
    print("Temps de connexion à la base de données redis:\n" + str(end_time - start_time) + " secondes.")

    ##################### IMPORT DES DONNEES #####################

    # Import des données sqlite dans redis
    start_time = datetime.now().timestamp()
    con_init = sqlite3.connect("mental_health.sqlite")
    end_time = datetime.now().timestamp()
    print("Temps de connexion à la base de données sqlite:\n" + str(end_time - start_time) + " secondes.")
    cur_init = con_init.cursor() 
    res = cur_init.execute("SELECT answer.SurveyID, survey.description, answer.QuestionID, question.questiontext, answer.AnswerText, answer.UserID from survey join answer on survey.SurveyId = answer.surveyId join question on question.QuestionID = answer.QuestionID LIMIT 1000")
    i = 0
    for row in res:
        r.set(str(i),str(row))
        i = i+1


    create_sqlite_database(con_init, cur_init)
    con_init.close()
    con = sqlite3.connect("mental_health_v2.sqlite")
    cur = con.cursor()
    

    #################################################
    ##################### REDIS #####################
    #################################################

    # Insertion de nb_data données dans redis
    pipeline = r.pipeline()
    for i in range(1001, 1001+nb_data+1):
        # Utilisation d'une donnée existant réellement dans la base de données
        pipeline.set(i, "(2016, 'mental health survey for 2016', 25, 'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?', 'None did', 2141)")
    
    start_time = datetime.now().timestamp()
    pipeline.execute()
    end_time = datetime.now().timestamp()
    insert_redis = (end_time - start_time)

    # Récupération de nb_data données dans redis
    pipeline = r.pipeline()
    for i in range(1001, 1001+nb_data+1):
        pipeline.get(i)

    start_time = datetime.now().timestamp()
    pipeline.execute()
    end_time = datetime.now().timestamp()
    retrieval_redis = (end_time - start_time)

    # Mise à jour de nb_data données dans redis
    pipeline = r.pipeline()
    for i in range(1001, 1001+nb_data+1):
        # Modification d'une donnée existant réellement dans la base de données
        pipeline.set(i, "(2015, 'mental health survey for 2015', 999, 'Have you ever studied computer science?', 'None did', 2141)")
    
    start_time = datetime.now().timestamp()
    pipeline.execute()
    end_time = datetime.now().timestamp()
    update_redis = (end_time - start_time)

    # Suppression de nb_data données dans redis
    pipeline = r.pipeline()
    for i in range(1001, 1001+nb_data+1):
        pipeline.delete(i)

    start_time = datetime.now().timestamp()
    pipeline.execute()
    end_time = datetime.now().timestamp()
    delete_redis = (end_time - start_time)

    ##################################################
    ##################### SQLITE #####################
    ##################################################
    
    # Insertion de nb_data données dans sqlite
    query_question = ""
    query_answer = ""
    for i in range(1001, 1001+nb_data+1):
        query_question += "INSERT INTO question (questiontext, QuestionID) VALUES ('Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?', '" + str(i) + "');"
        query_answer += "INSERT INTO answer (AnswerText, SurveyID, UserID, QuestionID) VALUES ('None did', 2016, 2141, '" + str(i) + "');"
    
    start_time = datetime.now().timestamp()
    cur.executescript(query_question)
    con.commit()
    intermediate_time = datetime.now().timestamp()
    cur.executescript(query_answer)
    con.commit()
    end_time = datetime.now().timestamp()
    first_insert_sqlite = (intermediate_time - start_time)
    second_insert_sqlite = (end_time - intermediate_time)
    total_insert_sqlite = (((intermediate_time - start_time)+(end_time - intermediate_time)))

    # Récupération de nb_data données dans sqlite
    query_retrieval = []
    for i in range(1001, 1001+nb_data+1):
        query_retrieval.append("SELECT answer.SurveyID, survey.description, answer.QuestionID, question.questiontext, answer.AnswerText, answer.UserID from survey join answer on survey.SurveyId = answer.surveyId join question on question.QuestionID = answer.QuestionID and answer.QuestionID = " + str(i) + ";")

    results = []
    retrieval_sqlite = 0
    for query in query_retrieval:
        start_time = datetime.now().timestamp()
        cur.execute(query)
        result = cur.fetchall()
        end_time = datetime.now().timestamp()
        results.append(result)
        retrieval_sqlite += (end_time - start_time)

    # Mise à jour de nb_data données dans sqlite
    query_update_question = ""
    query_update_answer = ""
    for i in range(1001, 1001+nb_data+1):
        query_update_question += "UPDATE question SET questiontext = 'Have you ever studied computer science?', QuestionID = " + str(1001+nb_data+i+1) + " WHERE questiontext = 'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?' and QuestionID = " + str(i) + ";"
        query_update_answer += "UPDATE answer SET SurveyID = 2015, QuestionID = "+ str(1001+nb_data+i+1) + " WHERE SurveyID = 2016 AND UserID = 2141 AND AnswerText = 'None did' AND QuestionID = " + str(i) + ";"
    
    start_time = datetime.now().timestamp()
    cur.executescript(query_update_question)
    con.commit()
    intermediate_time = datetime.now().timestamp()
    cur.executescript(query_update_answer)
    con.commit()
    end_time = datetime.now().timestamp()
    first_update_sqlite = (intermediate_time - start_time)
    second_update_sqlite = (end_time - intermediate_time)
    total_update_sqlite = (((intermediate_time - start_time)+(end_time - intermediate_time)))


    # Suppression de nb_data données dans sqlite
    query_delete_question = ""
    query_delete_answer = ""
    for i in range(1001, 1001+nb_data+1):
        query_delete_question += ("DELETE FROM question where QuestionID = " + str(1001+nb_data+i+1) + ";")
        query_delete_answer += ("DELETE FROM answer where QuestionID = " + str(1001+nb_data+i+1) + ";") 
    
    start_time = datetime.now().timestamp()
    cur.executescript(query_delete_question)
    con.commit()
    intermediate_time = datetime.now().timestamp()
    cur.executescript(query_delete_answer)
    con.commit()
    end_time = datetime.now().timestamp()
    first_delete_sqlite = (intermediate_time - start_time)
    second_delete_sqlite = (end_time - intermediate_time)
    total_delete_sqlite = (((intermediate_time - start_time)+(end_time - intermediate_time)))
    
    con.close()

    r.flushall()

    return [[insert_redis, retrieval_redis, update_redis, delete_redis], 
            [total_insert_sqlite, first_insert_sqlite, second_insert_sqlite, retrieval_sqlite,
            total_update_sqlite, first_update_sqlite, second_insert_sqlite, total_delete_sqlite,
            first_delete_sqlite, second_delete_sqlite]]

In [4]:
def print_and_return_redis_perfs(performance, nb_data, i):
    """Print and return redis performance for an operation

    Args:
        performance (float): Time taken for the nb_data trial on one operation
        i (int): Describes the type of operation: 1 = insertion, 2 = retrieval, 3 = update, 4 (or else) = delete
    """

    operation = ""

    if i == 0:
        operation = "d'insertion"
    elif i==1:
        operation = "de récupération"
    elif i==2:
        operation = "de modification"
    else:
        operation = "de suppression"

    print(f"Temps {operation} de {nb_data} données sur une base de données préremplies 1000 données sur Redis:             {performance:.8f} s.")


In [5]:
def print_and_return_sqlite_perfs(total, first, second, nb_data, i):
    """Print and return SQLite mean and standard deviation for an operation

    Args:
        tab_total (float[]): Total time taken (= time taken on first and second table) for the nb_data trial on one operation
        tab_first (float[]): Time taken on the first table for the nb_data trial on one operation
        tab_second (float[]): Time taken on the second table for the nb_data trial on one operation
        i (int): Describes the type of operation: 1 = insertion, 2 = retrieval, 3 = update, 4 (or else) = delete
    """

    operation = ""
    new_line = ""
    if i == 0:
        operation = "d'insertion"
    elif i == 1:
        operation = "de modification"
    else:
        operation = "de suppression"
        new_line = "\n\n"

    print(f"Temps {operation} de {nb_data} données sur une base de données préremplies 1000 données sur SQLite:             {total:.8f} s.")
    print(f"Temps {operation} de {nb_data} données sur la 1ère table: {first:.8f} s.")
    print(f"Temps {operation} de {nb_data} données sur la 2ème table:  {second:.8f} s.{new_line}")
    

In [6]:
list_nb_data = [2000, 25000, 50000, 75000, 100000, 125000, 150000, 175000, 200000]

redis_insert = []
redis_retrieval = []
redis_update = []
redis_delete = []

sqlite_insert = []
sqlite_retrieval = []
sqlite_update = []
sqlite_delete = []

for data in list_nb_data:
    perfs = launch_all(data)
    redis_insert.append(perfs[0][0])
    redis_retrieval.append(perfs[0][1])
    redis_update.append(perfs[0][2])
    redis_delete.append(perfs[0][3])
    print_and_return_redis_perfs(perfs[0][0], data, 0)
    print_and_return_redis_perfs(perfs[0][1], data, 1)
    print_and_return_redis_perfs(perfs[0][2], data, 2)
    print_and_return_redis_perfs(perfs[0][3], data, 3)

    print(f"Temps de récupération en moyenne sur 1000 données sur SQLite:            {perfs[1][3]:.8f} s.")
    sqlite_retrieval.append(perfs[1][3])
    print_and_return_sqlite_perfs(perfs[1][0],perfs[1][1],perfs[1][2],data, 0)
    sqlite_insert.append(perfs[1][0])
    print_and_return_sqlite_perfs(perfs[1][4],perfs[1][5],perfs[1][6],data, 1)
    sqlite_update.append(perfs[1][4])
    print_and_return_sqlite_perfs(perfs[1][7],perfs[1][8],perfs[1][9], data, 2)
    sqlite_delete.append(perfs[1][7])

##################################################
OPERATIONS SUR 2000 DONNEES 
##################################################


Temps de connexion à la base de données redis:
0.00011396408081054688 secondes.
Temps de connexion à la base de données sqlite:
0.00025010108947753906 secondes.
Temps d'insertion de 2000 données sur une base de données préremplies 1000 données sur Redis:             0.01686478 s.
Temps de récupération de 2000 données sur une base de données préremplies 1000 données sur Redis:             0.01427388 s.
Temps de modification de 2000 données sur une base de données préremplies 1000 données sur Redis:             0.01699495 s.
Temps de suppression de 2000 données sur une base de données préremplies 1000 données sur Redis:             0.01234007 s.
Temps de récupération en moyenne sur 1000 données sur SQLite:            0.38809514 s.
Temps d'insertion de 2000 données sur une base de données préremplies 1000 données sur SQLite:             6.51443291 s.
Temps d

In [ ]:
data_redis = {
    'nb_data': list_nb_data,
    'Insertion': redis_insert,
    'Selection': redis_retrieval,
    'Mise a jour': redis_update,
    'Suppression': redis_delete
}

df_redis = pd.DataFrame(data=data_redis)
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_redis['nb_data'], y=df_redis['Insertion'],
                         line=dict(color='blue'), mode='lines', name='Insertion'))

fig.add_trace(go.Scatter(x=df_redis['nb_data'], y=df_redis['Suppression'],
                         line=dict(color='red'), mode='lines', name='Suppression'))

fig.add_trace(go.Scatter(x=df_redis['nb_data'], y=df_redis['Mise a jour'],
                         line=dict(color='green'), mode='lines', name='Mise a jour'))

fig.add_trace(go.Scatter(x=df_redis['nb_data'], y=df_redis['Selection'],
                         line=dict(color='purple'), mode='lines', name='Récupération'))

fig.update_layout(
    title='Performance evolution with redis',
    xaxis_title='nb_data',
    yaxis_title='Values',
)


pio.write_html(fig, file='graph_redis.html', auto_open=True)

In [ ]:
data_sqlite = {
    'nb_data': list_nb_data,
    'Insertion': sqlite_insert, 
    'Selection': sqlite_retrieval, 
    'Mise a jour': sqlite_update, 
    'Suppression': sqlite_delete
}


df_sqlite = pd.DataFrame(data=data_sqlite)
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_sqlite['nb_data'], y=df_sqlite['Insertion'],
                         line=dict(color='blue'), mode='lines', name='Insertion'))

fig.add_trace(go.Scatter(x=df_sqlite['nb_data'], y=df_sqlite['Suppression'],
                         line=dict(color='red'), mode='lines', name='Suppression'))

fig.add_trace(go.Scatter(x=df_sqlite['nb_data'], y=df_sqlite['Mise a jour'],
                         line=dict(color='green'), mode='lines', name='Mise a jour'))

fig.add_trace(go.Scatter(x=df_sqlite['nb_data'], y=df_sqlite['Selection'],
                         line=dict(color='purple'), mode='lines', name='Récupération'))

fig.update_layout(
    title='Performance evolution with SQLite',
    xaxis_title='nb_data',
    yaxis_title='Values',
)

pio.write_html(fig, file='graph_sqlite.html', auto_open=True)
